In [1]:
## Testing olmo model
from PIL import Image
from transformers import AutoProcessor
from transformers import Qwen2VLForConditionalGeneration

In [ ]:
import torch
model_path = "allenai/olmOCR-7B-0225-preview"

model = Qwen2VLForConditionalGeneration.from_pretrained(
    model_path, 
    torch_dtype= torch.float32,
    device_map="auto",)
model.eval()

In [3]:
processor = AutoProcessor.from_pretrained(model_path, use_fast=True)

You have video processor config saved in `preprocessor.json` file which is deprecated. Video processor configs should be saved in their own `video_preprocessor.json` file. You can rename the file or load and save the processor back which renames it automatically. Loading from `preprocessor.json` will be removed in v5.0.


In [4]:
from transformers.image_utils import load_image
image_path = "../data/test1.jpg"
image = load_image(image_path)

In [5]:
prompt = """Extract all text and tables from this document image."""
messages = [
                {
                    "role": "user",
                    "content": [
                        {"type": "image", "image": image},
                        {"type": "text", "text": prompt},
                    ],
                }
]

chat_prompt = processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
inputs = processor(text=[chat_prompt], images=[image], padding=True, return_tensors="pt")

In [6]:
inputs = {k: v.to(model.device) for k, v in inputs.items()}

In [7]:
# Allow configurable generation parameters
gen_params = {
                "temperature": 0.8,
                "max_new_tokens": 15000,
                "num_return_sequences": 1,
                "do_sample": True,
            }

In [8]:
with torch.no_grad():
    output = model.generate(**inputs, **gen_params)

In [9]:
processor.tokenizer.batch_decode(output)

['<|im_start|>system\nYou are a helpful assistant.<|im_end|>\n<|im_start|>user\n<|vision_start|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image

In [10]:
inputs["input_ids"]

tensor([[151644,   8948,    198,  ..., 151644,  77091,    198]],
       device='cuda:0')

In [11]:
prompt_length = inputs["input_ids"].shape[1]

In [12]:
trimmed_generated_ids = output[:, prompt_length:]

In [13]:
output_sentence = processor.batch_decode(trimmed_generated_ids,skip_special_tokens=True)[0].lstrip()

In [14]:
print(output_sentence)

Again, we say “All over the world” when we should say “Over all the world,” and “the reason why” instead of “the reason that.” Usage has made our language what it is; grammatical rules strive to limit it to what it ought to be. In many instances usage has supplanted grammatical rules. Hundreds of words have been used by masters of English in ways that violate these rules. These uses are to be found to-day recorded by the dictionaries because lexicographers recognize it is their duty to present the language as they find it used by the people. It is to the people, not to the purists, that one must look for the enriching of our mother tongue. To them it is as impossible to confine the English language within the bonds of grammatical rules as it is to stem the tide of the sea. For them all matters that relate to English speech can be decided only by the law of good usage. This, and this alone is their Court of Last Resort. Withal, the observance of certain conventional rules does no harm i